In [86]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

## Salary_Class 확인 및 결측값, 중복값 제거 

In [87]:
data = pd.read_csv('salary_class.csv', index_col=0)
data.head(10)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,0
5,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,0
6,29,NaN,HS-grad,9,Never-married,NaN,Unmarried,Black,Male,0,0,40,United-States,0
7,63,Self-emp-not-inc,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,1
8,24,Private,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,0
9,55,Private,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,0


In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   education       48842 non-null  object
 3   education-num   48842 non-null  int64 
 4   marital-status  48842 non-null  object
 5   occupation      46033 non-null  object
 6   relationship    48842 non-null  object
 7   race            48842 non-null  object
 8   sex             48842 non-null  object
 9   capital-gain    48842 non-null  int64 
 10  capital-loss    48842 non-null  int64 
 11  hours-per-week  48842 non-null  int64 
 12  native-country  47985 non-null  object
 13  class           48842 non-null  int64 
dtypes: int64(6), object(8)
memory usage: 5.6+ MB


In [89]:
data.isnull().sum()

age                  0
workclass         2799
education            0
education-num        0
marital-status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     857
class                0
dtype: int64

In [90]:
# workclass에 2800, occupation에 2800, native-country에 850개 가량 NaN 값이 있는데 셋 중 하나라도 NaN이 있는 행의 개수는 총 3620개이다
len(data[data['workclass'].isnull() | data['occupation'].isnull() | data['native-country'].isnull()])

3620

In [91]:
# 결측값들을 최빈값으로 채우는 것은 workclass나 occupation, native-country의 특성상 너무 자의적이라 판단된다
# 또한 결측값이 약 48000개 중 3500 정도이니 그냥 해당 값들을 drop시키는 것이 나을 것이라 판단된다

data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45222 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             45222 non-null  int64 
 1   workclass       45222 non-null  object
 2   education       45222 non-null  object
 3   education-num   45222 non-null  int64 
 4   marital-status  45222 non-null  object
 5   occupation      45222 non-null  object
 6   relationship    45222 non-null  object
 7   race            45222 non-null  object
 8   sex             45222 non-null  object
 9   capital-gain    45222 non-null  int64 
 10  capital-loss    45222 non-null  int64 
 11  hours-per-week  45222 non-null  int64 
 12  native-country  45222 non-null  object
 13  class           45222 non-null  int64 
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


In [92]:
# 중복된 값의 개수를 확인
data.duplicated().sum()

5982

In [93]:
# 중복된 값들 중 첫 번째만 남기고 제거한 후 변화를 반영
data.drop_duplicates(keep='first', inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39240 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             39240 non-null  int64 
 1   workclass       39240 non-null  object
 2   education       39240 non-null  object
 3   education-num   39240 non-null  int64 
 4   marital-status  39240 non-null  object
 5   occupation      39240 non-null  object
 6   relationship    39240 non-null  object
 7   race            39240 non-null  object
 8   sex             39240 non-null  object
 9   capital-gain    39240 non-null  int64 
 10  capital-loss    39240 non-null  int64 
 11  hours-per-week  39240 non-null  int64 
 12  native-country  39240 non-null  object
 13  class           39240 non-null  int64 
dtypes: int64(6), object(8)
memory usage: 4.5+ MB


## 범주형 데이터 처리

In [94]:
# object형의 칼럼, 즉 범주형 데이터를 파악
categorical = [column for column in data.columns if data[column].dtype=='O'] 

print('num of categorical var: %d' % len(categorical))
print(categorical)

num of categorical var: 8
['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


In [95]:
for column in categorical:
    print(data[column].value_counts(), '\n')

workclass
Private             27717
Self-emp-not-inc     3669
Local-gov            2975
State-gov            1892
Self-emp-inc         1595
Federal-gov          1371
Without-pay            21
Name: count, dtype: int64 

education
HS-grad         12003
Some-college     8457
Bachelors        6522
Masters          2357
Assoc-voc        1859
Assoc-acdm       1470
11th             1422
10th             1136
7th-8th           801
Prof-school       764
9th               658
12th              540
Doctorate         526
5th-6th           438
1st-4th           217
Preschool          70
Name: count, dtype: int64 

marital-status
Married-civ-spouse       17947
Never-married            12169
Divorced                  5885
Separated                 1394
Widowed                   1261
Married-spouse-absent      552
Married-AF-spouse           32
Name: count, dtype: int64 

occupation
Prof-specialty       5516
Exec-managerial      5311
Adm-clerical         4742
Sales                4671
Craft-repair   

In [96]:
df = data.drop(categorical, axis=1)
df.head()

,age,education-num,capital-gain,capital-loss,hours-per-week,class
0,25,7,0,0,40,0
1,38,9,0,0,50,0
2,28,12,0,0,40,1
3,44,10,7688,0,40,1
5,34,6,0,0,30,0


In [98]:
df = df.drop('class', axis=1)
df.head()

,age,education-num,capital-gain,capital-loss,hours-per-week
0,25,7,0,0,40
1,38,9,0,0,50
2,28,12,0,0,40
3,44,10,7688,0,40
5,34,6,0,0,30


In [101]:
categ_data = pd.get_dummies(data[categorical], dtype=int)
categ_data.head()

,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [102]:
# One Hot Encoding된 categ_data를 수치형 데이터들만 남긴 df와 연결하여 전처리가 완료된 data를 만든다
new_data = pd.concat([df, categ_data], axis=1)
new_data.head()

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,7,0,0,40,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,9,0,0,50,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,12,0,0,40,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,10,7688,0,40,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,6,0,0,30,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


# 교차 검증 

In [103]:
# 기존 전처리된 data에서 label에 해당하는 class 열을 가져온다
label = data['class']
label.head()

0    0
1    0
2    1
3    1
5    0
Name: class, dtype: int64

In [106]:
X_train, X_test, y_train, y_test = train_test_split(new_data, label, test_size=0.2, random_state=111)

In [107]:
model = DecisionTreeClassifier()
model_sp1 = DecisionTreeClassifier(random_state=111, max_depth=3)

#모델 학습
model.fit(X_train, y_train) 
model_sp1.fit(X_train, y_train) 

# X train 데이터로 예측 --> test 데이터로 예측
pred_train = model.predict(X_train) # train 예측
pred_test = model.predict(X_test) # test 예측

pred_train_sp = model_sp1.predict(X_train) # train 예측
pred_test_sp = model_sp1.predict(X_test) # test 예측

print("의사결정나무 예측 정확도 점수")
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test), '\n')

print("의사결정나무 w/ max_depth=3 예측 정확도 점수")
print(accuracy_score(y_train, pred_train_sp))
print(accuracy_score(y_test, pred_test_sp), '\n')

의사결정나무 예측 정확도 점수
0.9793577981651376
0.7966360856269113 

의사결정나무 w/ max_depth=3 예측 정확도 점수
0.8321546890927625
0.8448012232415902 



<일반 의사결정 나무 모델 vs max_depth=3인 의사결정 나무 모델>   
1. 일반적인 의사결정나무 모델을 학습하였을 때,, train 데이터에 대해 0.98에 가까운 예측도를 갖는다 --> 학습 데이터에 완벽하게 적합되었다(과적합)
2. max_depth를 3으로 한 의사결정 나무 모델을 학습하였을 때, train데이터에 대해 0.83의 예측도를 갖는다 --> 학습 데이터에 대해 일반적인 의사결정 나무보다 낮은 예측도
3. 학습되지 않은 test 데이터에 대한 예측 정확도는 비교적 과적합이 덜한 max_depth를 3으로 한 의사결정 나무 모델이 더 높았다 (0.84 > 0.79)

## KFold로 과적합 방지가 가능할까? 

In [108]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5) # n_splits: 데이터셋을 분할할 세트의 개수 --> 1세트만 test 데이터, 나머지는 train 데이터가 된다

cv_accuracy_train=[]    # train 데이터의 예측 정확도를 저장할 빈 리스트
cv_accuracy_test=[]     # test 데이터의 예측 정확도를 저장할 빈 리스트
kf_model = DecisionTreeClassifier(random_state=111, max_depth=3)

In [109]:
n_iter = 0 # for문의 반복 횟수를 나타내며 초기값으로 0을 설정

for train_idx, test_idx in kfold.split(new_data):
    X_train, X_test = new_data.iloc[train_idx], new_data.iloc[test_idx]
    y_train, y_test = label.iloc[train_idx], label.iloc[test_idx]
    
    # 모델 학습
    kf_model.fit(X_train, y_train)
    
    # 예측
    kf_pred_train = kf_model.predict(X_train)
    df_pred_test = kf_model.predict(X_test)
    
    # 정확도를 계산하고 소수점 4자리로 반올림 
    accuracy_train = np.round(accuracy_score(y_train, kf_pred_train),4) 
    accuracy_test = np.round(accuracy_score(y_test, df_pred_test),4)
    # 해당하는 정확도 값을 해당 리스트에 append
    cv_accuracy_train.append(accuracy_train)
    cv_accuracy_test.append(accuracy_test)
    
    n_iter +=1 # 다음 시행을 위한 업데이트
    print('\n{} 번 train 교차 검증 정확도 :{} ,test 교차 검증 정확도 :{} '.format(n_iter,accuracy_train, accuracy_test))

print()
print('train 평균 정확도',round(np.mean(cv_accuracy_train), 4), 'test 평균 정확도',round(np.mean(cv_accuracy_test), 4))


1 번 train 교차 검증 정확도 :0.8347 ,test 교차 검증 정확도 :0.8347 

2 번 train 교차 검증 정확도 :0.8343 ,test 교차 검증 정확도 :0.8363 

3 번 train 교차 검증 정확도 :0.8354 ,test 교차 검증 정확도 :0.8315 

4 번 train 교차 검증 정확도 :0.8342 ,test 교차 검증 정확도 :0.8336 

5 번 train 교차 검증 정확도 :0.8344 ,test 교차 검증 정확도 :0.8327 

train 평균 정확도 0.8346 test 평균 정확도 0.8338


<KFold의 사용 유무에 따른 예측 정확도 차이 파악>
1. KFold를 사용한 의사결정 모델은 max_depth도 3으로 설정하였기에, 0.83(학습 예측), 0.84(테스트 예측)과 비교해야 한다
2. KFold를 사용한 의사결정 모델의 예측 정확도는 0.83(학습 에측), 0.83(테스트 예측)이 나왔다
3. 과적합 문제가 크게 개선되어 보이지 않고 거의 비슷한 수준의 예측 정확도가 관찰되었다. 

## SKFold로 과적합 방지가 가능할까? 

In [111]:
from sklearn.model_selection import StratifiedKFold

skf_salary = StratifiedKFold(n_splits=5)      # skFold

skf_cv_accuracy_train = []                  # skfold를 통해 train 데이터로 예측한 정확도를 저장할 빈 리스트
skf_cv_accuracy_test = []                   # skfold를 통해 test 데이터로 예측한 정확도를 저장할 빈 리스트

skf_model = DecisionTreeClassifier(random_state=111, max_depth=3)   # depth가 3인 의사결정 나무 모델 설정

In [112]:
n_iter=0

for train_idx, test_idx in skf_salary.split(new_data, label):     # skfold의 경우 split에 feature와 label이 둘 다 들어가야 한다
    X_train, X_test = new_data.iloc[train_idx], new_data.iloc[test_idx]
    y_train, y_test = label.iloc[train_idx], label.iloc[test_idx]
    
    # 학습
    skf_model.fit(X_train, y_train)
    # 예측
    skf_pred_train = skf_model.predict(X_train)
    skf_pred_test = skf_model.predict(X_test)
    
    # 정확도를 계산하고 소수점 4자리로 반올림 
    accuracy_train = np.round(accuracy_score(y_train, skf_pred_train),4)
    accuracy_test = np.round(accuracy_score(y_test, skf_pred_test),4)
    # 해당하는 정확도 값을 해당 리스트에 append
    skf_cv_accuracy_train.append(accuracy_train)
    skf_cv_accuracy_test.append(accuracy_test)
    
    n_iter +=1 # 다음 시행을 위한 업데이트
    print('\n{} 번 train 교차 검증 정확도 :{} ,test 교차 검증 정확도 :{} '.format(n_iter,accuracy_train, accuracy_test))
    
print()
print('train 평균 정확도',round(np.mean(skf_cv_accuracy_train), 4), 'test 평균 정확도',round(np.mean(skf_cv_accuracy_test), 4))


1 번 train 교차 검증 정확도 :0.8356 ,test 교차 검증 정확도 :0.831 

2 번 train 교차 검증 정확도 :0.8344 ,test 교차 검증 정확도 :0.836 

3 번 train 교차 검증 정확도 :0.836 ,test 교차 검증 정확도 :0.8293 

4 번 train 교차 검증 정확도 :0.8337 ,test 교차 검증 정확도 :0.8354 

5 번 train 교차 검증 정확도 :0.8333 ,test 교차 검증 정확도 :0.837 

train 평균 정확도 0.8346 test 평균 정확도 0.8337


<SKFold의 사용 유무에 따른 예측 정확도 차이 파악>
1. SKFold를 사용한 의사결정 모델도 max_depth도 3으로 설정하였으며 split도 5로 설정하여 기존 모델들과 다른 부분들은 일치시켰다
2. 비교 대상으로 Kfold를 쓰지 않은 경우(0.83(학습 예측), 0.84(테스트 예측)) Kfold를 쓴 경우(0.83(학습 에측), 0.83(테스트 예측))이 있다
3. SKFold를 사용한 의사결정 모델의 예측 정확도는 0.83(학습 에측), 0.83(테스트 예측)이 나왔다
4. SKFold를 사용한 경우도 과적합 문제가 크게 개선되어 보이지 않고 이전 모델들과 거의 비슷한 수준의 예측 정확도가 관찰되었다.

## 결론
#### iris, titanic, salary_class로 데이터의 크기가 커졌음에도, K-Fold와 SK-Fold는 과적합 문제를 해결하지 못한다는 것을 확인할 수 있었다.